## Project 02 - Data wrangling and exploration using pandas
Working with the data to replicate the Table 3 of the paper. In the table 3, we are interested 4 items.
- HbA1C
- Admission Source
- Specialty
- Primary Diagnosis

Create 4 data frames for each of the items mentioned with the columns and rows match the table 3 corresponding to the item.

### Package import and data preprocess
1. The preliminary dataset contained multiple inpatient visits for some patients and the observations could not be considered as statistically independent, an assumption of the logistic regression model. We thus used only one encounter per patient; in particular, we considered only the first encounter for each patient as the primary admission and determined whether or not they were readmitted within 30 days. 
2. Additionally, we removed all encounters that resulted in either discharge to a hospice or patient death, to avoid biasing our analysis. From "IDs_mapping.csv" find that for "discharge_disposition_id", if the code is 11, 13, 14, 19, 20 and 21 then this patient discharge to a hospice or death.

After performing the above-described operations, we were left with 69973 encounters that constituted the final dataset for analysis.

In [1]:
import pandas as pd
df = pd.read_csv('diabetic_data.csv')
#df.info()

In [2]:
#1.Filter only the first encounter for each patient
pt = df.groupby('patient_nbr')['encounter_id'].first().reset_index()
df_pt = df[df['encounter_id'].isin(pt['encounter_id'].drop_duplicates().tolist())]
print(len(df_pt)) #length is 71518 now, means there are 71518 individual patients

#2. Remove all encounters that resulted in discharge to a hospice 
data = df_pt[(df_pt['discharge_disposition_id'] != 11) & (df_pt['discharge_disposition_id'] != 13)
           & (df_pt['discharge_disposition_id'] != 14) & (df_pt['discharge_disposition_id'] != 19)
           & (df_pt['discharge_disposition_id'] != 20) & (df_pt['discharge_disposition_id'] != 21)]
print(len(data)) #length is 69973 now.

71518
69973


Possible way to get 69984 patients:

- Remove encounters that resulted in discharge to a hospice first and then filter the first encounter for each patient. This can cuase some patients with not only one encounters and died at end to be filtered in the first step but actually their first admission should be counted.
- When removing hospice encounters, removed encounters with code 10 in "discharge_disposition_id" which contains 6 encounters, could be a mistake from the authors (I'm not sure with this one but this is the only way for me to get 69984).

Meanwhile, as my population number is different with the paper, all of the numbers below are not the same when compare with the paper, but most of them are approach. This occurs because the way I used to get the population was different with the paper.

In [3]:
df_f = df[(df['discharge_disposition_id'] != 11) & (df['discharge_disposition_id'] != 13)
            & (df['discharge_disposition_id'] != 14) & (df['discharge_disposition_id'] != 19)
            & (df['discharge_disposition_id'] != 20) & (df['discharge_disposition_id'] != 21)
            & (df['discharge_disposition_id'] != 10)]
pt_f = df_f.groupby('patient_nbr')['encounter_id'].first().reset_index()
data_f = df_f[df_f['encounter_id'].isin(pt_f['encounter_id'].drop_duplicates().tolist())]
len(data_f)

69984

### HbA1C
#### Row index
- Num_encounter （Number of encounters）: count of encounter_id
- Percentage (% of the population): count of patient_nbr for (No test was performed, Result was high and the diabetic medication was changed, Result was high but the diabetic medication was not changed, Normal result of the test) / count of patient_nbr
- Readm_num_encounter (Readmitted Number of encounters): count of "<30"and ">30" values of readmitted encounter_id
- Readm_percentage (Readmitted % in group): count of "<30"and ">30" values of readmitted patient_nbr for (No test was performed, Result was high and the diabetic medication was changed, Result was high but the diabetic medication was not changed, Normal result of the test) / count of "<30"and ">30" values of readmitted patient_nbr

#### Column index
- No_test (No test was performed): "None" value of A1Cresult
- High_change (Result was high and the diabetic medication was changed): “>8” value of A1Cresult, “Ch” value of change
- High_no_change (Result was high but the diabetic medication was not changed): “>8” value of A1Cresult, “No” value of change
- Normal (Normal result of the test): “Norm” value of A1Cresult

In [4]:
hb = data[['encounter_id','patient_nbr','A1Cresult','change','readmitted']]
#hb

In [5]:
hb['readmitted'].value_counts()

NO     41474
>30    22222
<30     6277
Name: readmitted, dtype: int64

The result of readmitted patients with no test results in the paper is 5342, which is similar to 6277, so they may defined readmitted as readmitted only less than 30 days. Here I use this definition for readmitted patients.

In [6]:
hba1c = pd.DataFrame(columns=['Num_encounter', 'Percentage', 'Readm_num_encounter','Readm_percentage'], 
                     index=['No_test', 'High_change', 'High_no_change','Normal'])

#The first row of HbA1C table
hb11 = hb[['encounter_id','readmitted']][hb['A1Cresult'] == 'None'].drop_duplicates() 
hb13 = hb11['encounter_id'][hb11['readmitted'] == '<30'].drop_duplicates()
hba1c.iloc[0][0] = len(hb11)
hba1c.iloc[0][1] = str(round(len(hb11) / len(hb),3) * 100) + '%'
hba1c.iloc[0][2] = len(hb13)
hba1c.iloc[0][3] = str(round(len(hb13) / len(hb11) * 100,1)) + '%'

#The second row of HbA1C table
hb21 = hb[['encounter_id','readmitted']][hb['A1Cresult'] == '>8'][hb['change'] == 'Ch'].drop_duplicates() 
hb23 = hb21['encounter_id'][hb21['readmitted'] == '<30'].drop_duplicates()
hba1c.iloc[1][0] = len(hb21)
hba1c.iloc[1][1] = str(round(len(hb21) / len(hb) * 100,2)) + '%'
hba1c.iloc[1][2] = len(hb23)
hba1c.iloc[1][3] = str(round(len(hb23) / len(hb21) * 100,1)) + '%'

#The third row of HbA1C table
hb31 = hb[['encounter_id','readmitted']][hb['A1Cresult'] == '>8'][hb['change'] == 'No'].drop_duplicates()
hb33 = hb31['encounter_id'][hb31['readmitted'] == '<30'].drop_duplicates()
hba1c.iloc[2][0] = len(hb31)
hba1c.iloc[2][1] = str(round(len(hb31) / len(hb) * 100,1)) + '%'
hba1c.iloc[2][2] = len(hb33)
hba1c.iloc[2][3] = str(round(len(hb33) / len(hb31) * 100,1)) + '%'

/Users/christy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]
/Users/christy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [7]:
hb['A1Cresult'].value_counts()

None    57128
>8       6239
Norm     3741
>7       2865
Name: A1Cresult, dtype: int64

The result of normal result of the test in the paper is 6637, which is similar to 3741 + 2865 = 6606, so they may defined normal results as test results with normal and >7 results. Here I use 6606.

In [8]:
#The last row of HbA1C table
hb41 = hb[['encounter_id','readmitted']][(hb['A1Cresult'] == 'Norm') | (hb['A1Cresult'] == '>7')].drop_duplicates()
hb43 = hb41['encounter_id'][hb41['readmitted'] == '<30'].drop_duplicates()
hba1c.iloc[3][0] = len(hb41)
hba1c.iloc[3][1] = str(round(len(hb41) / len(hb) * 100,1)) + '%'
hba1c.iloc[3][2] = len(hb43)
hba1c.iloc[3][3] = str(round(len(hb43) / len(hb41) * 100,1)) + '%'

#### Result -- HbA1C Table

In [9]:
hba1c

,Num_encounter,Percentage,Readm_num_encounter,Readm_percentage
No_test,57128,81.6%,5199,9.1%
High_change,4058,5.8%,348,8.6%
High_no_change,2181,3.1%,161,7.4%
Normal,6606,9.4%,569,8.6%


### Admission source
#### Row index
- Num_encounter （Number of encounters）: count of encounter_id
- Percentage (% of the population): count of patient_nbr for (No test was performed, Result was high and the diabetic medication was changed, Result was high but the diabetic medication was not changed, Normal result of the test) / count of patient_nbr
- Readm_num_encounter (Readmitted Number of encounters): count of "<30"and ">30" values of readmitted encounter_id
- Readm_percentage (Readmitted % in group): count of "<30"and ">30" values of readmitted patient_nbr for (No test was performed, Result was high and the diabetic medication was changed, Result was high but the diabetic medication was not changed, Normal result of the test) / count of "<30"and ">30" values of readmitted patient_nbr

#### Column index
- Emer (Admitted from emergency room): with code 7 in admission_source_id
- Ref (Admitted because of physician/clinic referral): with code 1, 2 in admission_source_id
- Other (Otherwise): with code not equal to 1, 2 and 7 in admission_source_id

In [10]:
ads = data[['encounter_id','patient_nbr','admission_source_id','change','readmitted']]
#ads

In [11]:
#ads['admission_source_id'].value_counts()

In [12]:
adm_s = pd.DataFrame(columns=['Num_encounter', 'Percentage', 'Readm_num_encounter','Readm_percentage'], 
                     index=['Emer', 'Ref', 'Other'])

ads11 = ads[['encounter_id','readmitted']][ads['admission_source_id'] == 7].drop_duplicates()
ads13 = ads11['encounter_id'][ads11['readmitted'] == '<30'].drop_duplicates()
adm_s.iloc[0][0] = len(ads11)
adm_s.iloc[0][1] = str(round(len(ads11) / len(ads),3) * 100) + '%'
adm_s.iloc[0][2] = len(ads13)
adm_s.iloc[0][3] = str(round(len(ads13) / len(ads11) * 100,1)) + '%'

ads21 = ads[['encounter_id','readmitted']][(ads['admission_source_id'] == 1) | (ads['admission_source_id'] == 2)].drop_duplicates()
ads23 = ads21['encounter_id'][ads21['readmitted'] == '<30'].drop_duplicates()
adm_s.iloc[1][0] = len(ads21)
adm_s.iloc[1][1] = str(round(len(ads21) / len(ads),3) * 100) + '%'
adm_s.iloc[1][2] = len(ads23)
adm_s.iloc[1][3] = str(round(len(ads23) / len(ads21) * 100,1)) + '%'

ads31 = ads[['encounter_id','readmitted']][(ads['admission_source_id'] != 1) 
                                           & (ads['admission_source_id'] != 2) & (ads['admission_source_id'] != 7)].drop_duplicates()
ads33 = ads31['encounter_id'][ads31['readmitted'] == '<30'].drop_duplicates()
adm_s.iloc[2][0] = len(ads31)
adm_s.iloc[2][1] = str(round(len(ads31) / len(ads) * 100,1)) + '%'
adm_s.iloc[2][2] = len(ads33)
adm_s.iloc[2][3] = str(round(len(ads33) / len(ads31) * 100,1)) + '%'

#### Result -- Admission Source Table

In [13]:
adm_s

,Num_encounter,Percentage,Readm_num_encounter,Readm_percentage
Emer,37260,53.2%,3446,9.2%
Ref,22654,32.4%,1952,8.6%
Other,10059,14.4%,879,8.7%


### Specialty
#### Row index
- Num_encounter （Number of encounters）: count of encounter_id
- Percentage (% of the population): count of patient_nbr for (No test was performed, Result was high and the diabetic medication was changed, Result was high but the diabetic medication was not changed, Normal result of the test) / count of patient_nbr
- Readm_num_encounter (Readmitted Number of encounters): count of "<30"and ">30" values of readmitted encounter_id
- Readm_percentage (Readmitted % in group): count of "<30"and ">30" values of readmitted patient_nbr for (No test was performed, Result was high and the diabetic medication was changed, Result was high but the diabetic medication was not changed, Normal result of the test) / count of "<30"and ">30" values of readmitted patient_nbr

#### Column index
- In_med (Internal Medicine): "InternalMedicine" value of medical_specialty
- Card (Cardiology): “Cardiology” value of medical_specialty
- Surg (Surgery): all the values with "Surgery" in medical_specialty
- Fam_prac (Family/general practice): “Family/GeneralPractice” value of medical_specialty
- Missing (Missing or unknown):"?" value of medical_specialty
- Other (Other): Other values in medical_specialty

In [14]:
sp = data[['encounter_id','patient_nbr','medical_specialty','change','readmitted']]
#sp

In [15]:
#sp['medical_specialty'].value_counts()

In [16]:
spec = pd.DataFrame(columns=['Num_encounter', 'Percentage', 'Readm_num_encounter','Readm_percentage'], 
                     index=['In_med', 'Card', 'Surg', 'Fam_prac', 'Missing', 'Other'])

sp11 = sp[['encounter_id','readmitted']][sp['medical_specialty'] == 'InternalMedicine'].drop_duplicates()
sp13 = sp11['encounter_id'][sp11['readmitted'] == '<30'].drop_duplicates()
spec.iloc[0][0] = len(sp11)
spec.iloc[0][1] = str(round(len(sp11) / len(sp) * 100, 1)) + '%'
spec.iloc[0][2] = len(sp13)
spec.iloc[0][3] = str(round(len(sp13) / len(sp11) * 100,1)) + '%'

sp21 = sp[['encounter_id','readmitted']][sp['medical_specialty'] == 'Cardiology'].drop_duplicates()
sp23 = sp21['encounter_id'][sp21['readmitted'] == '<30'].drop_duplicates()
spec.iloc[1][0] = len(sp21)
spec.iloc[1][1] = str(round(len(sp21) / len(sp) * 100, 1)) + '%'
spec.iloc[1][2] = len(sp23)
spec.iloc[1][3] = str(round(len(sp23) / len(sp21) * 100,1)) + '%'

#There are many values start with "Surgery" but with different types of surgery
sp31 = sp[['encounter_id','readmitted']][sp['medical_specialty'].str.startswith('Surgery')].drop_duplicates()
sp33 = sp31['encounter_id'][sp31['readmitted'] == '<30'].drop_duplicates()
spec.iloc[2][0] = len(sp31)
spec.iloc[2][1] = str(round(len(sp31) / len(sp) * 100, 1)) + '%'
spec.iloc[2][2] = len(sp33)
spec.iloc[2][3] = str(round(len(sp33) / len(sp31) * 100,1)) + '%'

sp41 = sp[['encounter_id','readmitted']][sp['medical_specialty'] == 'Family/GeneralPractice'].drop_duplicates()
sp43 = sp41['encounter_id'][sp41['readmitted'] == '<30'].drop_duplicates()
spec.iloc[3][0] = len(sp41)
spec.iloc[3][1] = str(round(len(sp41) / len(sp) * 100, 1)) + '%'
spec.iloc[3][2] = len(sp43)
spec.iloc[3][3] = str(round(len(sp43) / len(sp41) * 100,1)) + '%'

sp51 = sp[['encounter_id','readmitted']][sp['medical_specialty'] == '?'].drop_duplicates()
sp53 = sp51['encounter_id'][sp51['readmitted'] == '<30'].drop_duplicates()
spec.iloc[4][0] = len(sp51)
spec.iloc[4][1] = str(round(len(sp51) / len(sp) * 100, 1)) + '%'
spec.iloc[4][2] = len(sp53)
spec.iloc[4][3] = str(round(len(sp53) / len(sp51) * 100,1)) + '%'

#Concatenate all the filters above to get others
sp_c = pd.concat([sp11, sp21, sp31, sp41, sp51]).dropna()
sp61 = sp[['encounter_id','readmitted']][~(sp['encounter_id'].isin(sp_c['encounter_id'].drop_duplicates().tolist()))]
sp63 = sp61['encounter_id'][sp61['readmitted'] == '<30'].drop_duplicates()
spec.iloc[5][0] = len(sp61)
spec.iloc[5][1] = str(round(len(sp61) / len(sp) * 100, 1)) + '%'
spec.iloc[5][2] = len(sp63)
spec.iloc[5][3] = str(round(len(sp63) / len(sp61) * 100,1)) + '%'

#### Result -- Specialty Table

In [17]:
spec

,Num_encounter,Percentage,Readm_num_encounter,Readm_percentage
In_med,10641,15.2%,1039,9.8%
Card,4207,6.0%,302,7.2%
Surg,3685,5.3%,293,8.0%
Fam_prac,4978,7.1%,485,9.7%
Missing,33639,48.1%,3104,9.2%
Other,12823,18.3%,1054,8.2%


### Primary Diagnosis
#### Row index
- Num_encounter （Number of encounters）: count of encounter_id
- Percentage (% of the population): count of patient_nbr for (No test was performed, Result was high and the diabetic medication was changed, Result was high but the diabetic medication was not changed, Normal result of the test) / count of patient_nbr
- Readm_num_encounter (Readmitted Number of encounters): count of "<30"and ">30" values of readmitted encounter_id
- Readm_percentage (Readmitted % in group): count of "<30"and ">30" values of readmitted patient_nbr for (No test was performed, Result was high and the diabetic medication was changed, Result was high but the diabetic medication was not changed, Normal result of the test) / count of "<30"and ">30" values of readmitted patient_nbr

#### Column index
- Circ_sys (A disease of the circulatory system (icd9: 390–459, 785)): with value 390-459 and 785 of diag_1
- Diabetes (Diabetes (icd9: 250.xx)): with value 250.xx of diag_1
- Resp_sys (A disease of the respiratory system (icd9: 460–519, 786)): with value 460–519 and 786 of diag_1
- Dige_sys (Diseases of the digestive system (icd9: 520–579, 787)): with value 520–579 and 787 of diag_1
- Inj_pois (Injury and poisoning (icd9: 800–999)): with value 800–999 of diag_1
- Musc_sys (Diseases of the musculoskeletal system and connective tissue (icd9: 710–739)): with value 710–739 of diag_1
- Geni_sys (Diseases of the genitourinary system (icd9: 580–629, 788)): with value 580–629 and 788 of diag_1
- Neoplasms (Neoplasms (icd9: 140–239)): with value 140–239 of diag_1
- Other (other): all other values of diag_1

In [18]:
pr = data[['encounter_id','patient_nbr','diag_1','change','readmitted']]
#pr

In [19]:
#Prepare for filter in the next step
pr_le = pr[(pr['diag_1'].str.startswith('V')) | (pr['diag_1'].str.startswith('E'))] #Values start with a letter
pr_na = pr[pr['diag_1'] == '?'] 
pr_fl = pr[pr['diag_1'].str.contains('\.')] #Values contain "."
pr1 = pd.concat([pr_le, pr_na, pr_fl]).dropna()
prim = pr[~(pr['encounter_id'].isin(pr1['encounter_id'].drop_duplicates().tolist()))]
prim['diag_1'] = prim['diag_1'].astype(int)

/Users/christy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [20]:
diag = pd.DataFrame(columns=['Num_encounter', 'Percentage', 'Readm_num_encounter','Readm_percentage'], 
                     index=['Circ_sys', 'Diabetes', 'Resp_sys', 'Dige_sys', 'Inj_pois', 'Musc_sys', 'Geni_sys', 'Neoplasms', 'Other'])

pr11 = prim[['encounter_id','readmitted']][((prim['diag_1'] >= 390) & (prim['diag_1'] <= 459)) 
                                           | (prim['diag_1'] == 785)].drop_duplicates()
pr13 = pr11['encounter_id'][pr11['readmitted'] == '<30'].drop_duplicates()
diag.iloc[0][0] = len(pr11)
diag.iloc[0][1] = str(round(len(pr11) / len(pr) * 100, 1)) + '%'
diag.iloc[0][2] = len(pr13)
diag.iloc[0][3] = str(round(len(pr13) / len(pr11) * 100,1)) + '%'

pr21 = pr[['encounter_id','readmitted']][pr['diag_1'].str.startswith('250')].drop_duplicates()
pr23 = pr21['encounter_id'][pr21['readmitted'] == '<30'].drop_duplicates()
diag.iloc[1][0] = len(pr21)
diag.iloc[1][1] = str(round(len(pr21) / len(pr) * 100, 1)) + '%'
diag.iloc[1][2] = len(pr23)
diag.iloc[1][3] = str(round(len(pr23) / len(pr21) * 100,1)) + '%'

pr31 = prim[['encounter_id','readmitted']][((prim['diag_1'] >= 460) & (prim['diag_1'] <= 519)) 
                                           | (prim['diag_1'] == 786)].drop_duplicates()
pr33 = pr31['encounter_id'][pr31['readmitted'] == '<30'].drop_duplicates()
diag.iloc[2][0] = len(pr31)
diag.iloc[2][1] = str(round(len(pr31) / len(pr) * 100, 1)) + '%'
diag.iloc[2][2] = len(pr33)
diag.iloc[2][3] = str(round(len(pr33) / len(pr31) * 100,1)) + '%'

pr41 = prim[['encounter_id','readmitted']][((prim['diag_1'] >= 520) & (prim['diag_1'] <= 579)) 
                                           | (prim['diag_1'] == 787)].drop_duplicates()
pr43 = pr41['encounter_id'][pr41['readmitted'] == '<30'].drop_duplicates()
diag.iloc[3][0] = len(pr41)
diag.iloc[3][1] = str(round(len(pr41) / len(pr) * 100, 1)) + '%'
diag.iloc[3][2] = len(pr43)
diag.iloc[3][3] = str(round(len(pr43) / len(pr41) * 100,1)) + '%'

pr51 = prim[['encounter_id','readmitted']][((prim['diag_1'] >= 800) & (prim['diag_1'] <= 999))].drop_duplicates()
pr53 = pr51['encounter_id'][pr51['readmitted'] == '<30'].drop_duplicates()
diag.iloc[4][0] = len(pr51)
diag.iloc[4][1] = str(round(len(pr51) / len(pr) * 100, 1)) + '%'
diag.iloc[4][2] = len(pr53)
diag.iloc[4][3] = str(round(len(pr53) / len(pr51) * 100,1)) + '%'

pr61 = prim[['encounter_id','readmitted']][((prim['diag_1'] >= 710) & (prim['diag_1'] <= 739))].drop_duplicates()
pr63 = pr61['encounter_id'][pr61['readmitted'] == '<30'].drop_duplicates()
diag.iloc[5][0] = len(pr61)
diag.iloc[5][1] = str(round(len(pr61) / len(pr) * 100, 1)) + '%'
diag.iloc[5][2] = len(pr63)
diag.iloc[5][3] = str(round(len(pr63) / len(pr61) * 100,1)) + '%'

pr71 = prim[['encounter_id','readmitted']][((prim['diag_1'] >= 580) & (prim['diag_1'] <= 629)) 
                                           | (prim['diag_1'] == 788)].drop_duplicates()
pr73 = pr71['encounter_id'][pr71['readmitted'] == '<30'].drop_duplicates()
diag.iloc[6][0] = len(pr71)
diag.iloc[6][1] = str(round(len(pr71) / len(pr) * 100, 1)) + '%'
diag.iloc[6][2] = len(pr73)
diag.iloc[6][3] = str(round(len(pr73) / len(pr71) * 100,1)) + '%'

pr81 = prim[['encounter_id','readmitted']][((prim['diag_1'] >= 140) & (prim['diag_1'] <= 239))].drop_duplicates()
pr83 = pr81['encounter_id'][pr81['readmitted'] == '<30'].drop_duplicates()
diag.iloc[7][0] = len(pr81)
diag.iloc[7][1] = str(round(len(pr81) / len(pr) * 100, 1)) + '%'
diag.iloc[7][2] = len(pr83)
diag.iloc[7][3] = str(round(len(pr83) / len(pr81) * 100,1)) + '%'

pr_c = pd.concat([pr11, pr21, pr31, pr41, pr51, pr61, pr71, pr81]).dropna()
pr91 = pr[['encounter_id','readmitted']][~(pr['encounter_id'].isin(pr_c['encounter_id'].drop_duplicates().tolist()))]
pr93 = pr91['encounter_id'][pr91['readmitted'] == '<30'].drop_duplicates()
diag.iloc[8][0] = len(pr91)
diag.iloc[8][1] = str(round(len(pr91) / len(pr) * 100, 1)) + '%'
diag.iloc[8][2] = len(pr93)
diag.iloc[8][3] = str(round(len(pr93) / len(pr91) * 100,1)) + '%'

#### Result -- Primary Diagnosis Table

In [21]:
diag

,Num_encounter,Percentage,Readm_num_encounter,Readm_percentage
Circ_sys,21384,30.6%,2067,9.7%
Diabetes,5748,8.2%,524,9.1%
Resp_sys,9486,13.6%,691,7.3%
Dige_sys,6487,9.3%,520,8.0%
Inj_pois,4694,6.7%,506,10.8%
Musc_sys,4064,5.8%,341,8.4%
Geni_sys,3440,4.9%,309,9.0%
Neoplasms,2538,3.6%,230,9.1%
Other,12132,17.3%,1089,9.0%
